In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import datetime
import seaborn as sns

#import pydicom
import time
from functools import partial
import gc
import operator 
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.utils.data as D
import torch.nn.functional as F
from sklearn.model_selection import KFold
from tqdm import tqdm, tqdm_notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')
import pickle
%load_ext autoreload
%autoreload 2
%matplotlib inline
from skimage.io import imread,imshow
from helper import *
import helper
import torchvision.models as models
from torch.optim import Adam
from defenitions import *

/home/reina/anaconda3/envs/RSNA/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/reina/anaconda3/envs/RSNA/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [2]:
SEED = 8153
device=device_by_name("Tesla")
#device=device_by_name("RTX")
#device=device_by_name("1060")
torch.cuda.set_device(device)
#device = "cpu"
sendmeemail=Email_Progress(my_gmail,my_pass,to_email,'Densenet161-Copy2-2 results')

In [3]:
def get_submission(test_df,pred):
    epidural_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_epidural','Label':torch.sigmoid(pred[:,0])})
    intraparenchymal_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_intraparenchymal','Label':torch.sigmoid(pred[:,1])})
    intraventricular_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_intraventricular','Label':torch.sigmoid(pred[:,2])})
    subarachnoid_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_subarachnoid','Label':torch.sigmoid(pred[:,3])})
    subdural_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_subdural','Label':torch.sigmoid(pred[:,4])})
    any_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_any','Label':torch.sigmoid(pred[:,5])}) 
    return pd.concat([epidural_df,
                        intraparenchymal_df,
                        intraventricular_df,
                        subarachnoid_df,
                        subdural_df,
                        any_df]).sort_values('ID').reset_index(drop=True)

In [4]:
def get_submission_ids(image_ids,pred,do_sigmoid=True):
    if do_sigmoid:
        func = lambda x:torch.sigmoid(x)
    else:
        func = lambda x:x
    epidural_df=pd.DataFrame(data={'ID':'ID_'+image_ids+'_epidural','Label':func(pred[:,0])})
    intraparenchymal_df=pd.DataFrame(data={'ID':'ID_'+image_ids+'_intraparenchymal','Label':func(pred[:,1])})
    intraventricular_df=pd.DataFrame(data={'ID':'ID_'+image_ids+'_intraventricular','Label':func(pred[:,2])})
    subarachnoid_df=pd.DataFrame(data={'ID':'ID_'+image_ids+'_subarachnoid','Label':func(pred[:,3])})
    subdural_df=pd.DataFrame(data={'ID':'ID_'+image_ids+'_subdural','Label':func(pred[:,4])})
    any_df=pd.DataFrame(data={'ID':'ID_'+image_ids+'_any','Label':func(pred[:,5])}) 
    return pd.concat([epidural_df,
                        intraparenchymal_df,
                        intraventricular_df,
                        subarachnoid_df,
                        subdural_df,
                        any_df]).sort_values('ID').reset_index(drop=True)

In [5]:
old_train_df = pd.read_csv(data_dir+'train.csv')
old_train_df.shape
old_train_df=old_train_df[~old_train_df.PatientID.isin(bad_images)].reset_index(drop=True)
old_train_df=old_train_df.drop_duplicates().reset_index(drop=True)
old_train_df.shape
old_train_df.head()

train_df = pd.read_csv(data_dir+'train_stage2.csv')
train_df.shape


old_test_df = pd.read_csv(data_dir+'test.csv')
old_test_df.head()
assert old_test_df.shape[0]+old_train_df.shape[0]==train_df.shape[0]


(674252, 15)

(674252, 15)

,PatientID,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,any,PID,StudyI,SeriesI,WindowCenter,WindowWidth,ImagePositionZ,ImagePositionX,ImagePositionY
0,63eb1e259,0,0,0,0,0,0,a449357f,62d125e5b2,0be5c0d1b3,"['00036', '00036']","['00080', '00080']",180.199951,-125.0,-8.000000
1,2669954a7,0,0,0,0,0,0,363d5865,a20b80c7bf,3564d584db,"['00047', '00047']","['00080', '00080']",922.530821,-156.0,45.572849
2,52c9913b1,0,0,0,0,0,0,9c2b4bd7,3e3634f8cf,973274ffc9,40,150,4.455000,-125.0,-115.063000
3,4e6ff6126,0,0,0,0,0,0,3ae81c2d,a1390c15c2,e5ccad8244,"['00036', '00036']","['00080', '00080']",100.000000,-99.5,28.500000
4,7858edd88,0,0,0,0,0,0,c1867feb,c73e81ed3a,28e0531b3a,40,100,145.793000,-125.0,-132.190000


(752797, 15)

,PatientID,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,any,SeriesI,PID,StudyI,WindowCenter,WindowWidth,ImagePositionZ,ImagePositionX,ImagePositionY
0,28fbab7eb,0.5,0.5,0.5,0.5,0.5,0.5,ebfd7e4506,cf1b6b11,93407cadbb,30,80,158.458000,-125.0,-135.598000
1,877923b8b,0.5,0.5,0.5,0.5,0.5,0.5,6d95084e15,ad8ea58f,a337baa067,30,80,138.729050,-125.0,-101.797981
2,a591477cb,0.5,0.5,0.5,0.5,0.5,0.5,8e06b2c9e0,ecfb278b,0cfe838d54,30,80,60.830002,-125.0,-133.300003
3,42217c898,0.5,0.5,0.5,0.5,0.5,0.5,e800f419cf,e96e31f4,c497ac5bad,30,80,55.388000,-125.0,-146.081000
4,a130c4d2f,0.5,0.5,0.5,0.5,0.5,0.5,faeb7454f3,69affa42,854e4fbc01,30,80,33.516888,-125.0,-118.689819


In [6]:
test_df = pd.read_csv(data_dir+'test_stage2.csv')
test_df.head()

,PatientID,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,any,SeriesI,PID,StudyI,WindowCenter,WindowWidth,ImagePositionZ,ImagePositionX,ImagePositionY
0,0fbf6a978,0.5,0.5,0.5,0.5,0.5,0.5,48eaa460c6,3556ae0e,3710aff1c5,40,100,37.690857,-125.0,-117.830185
1,d62ec3412,0.5,0.5,0.5,0.5,0.5,0.5,389cec2e11,c667d610,8e38d86ede,"['00036', '00036']","['00080', '00080']",762.200043,-66.0,-10.000000
2,cb544194b,0.5,0.5,0.5,0.5,0.5,0.5,a6ebe3ca4f,9389d3b6,7040866911,"['00036', '00036']","['00080', '00080']",818.400024,-105.0,6.000000
3,0d62513ec,0.5,0.5,0.5,0.5,0.5,0.5,01c388cfe3,79cdc6e4,07c298b5be,30,80,31.857132,-125.0,-112.984024
4,fc45b2151,0.5,0.5,0.5,0.5,0.5,0.5,d16ad143cb,39eb6eb6,6a15b35eae,"['00036', '00036']","['00080', '00080']",424.099976,-102.5,14.500000


In [7]:
def my_loss(y_pred,y_true,weights):
    window=(y_true>=0).to(torch.float)
    loss = (F.binary_cross_entropy_with_logits(y_pred,y_true,reduction='none')*window*weights.expand_as(y_true)).mean()/(window.mean()+1e-7)
    return loss

In [8]:
class Metric():
    def __init__(self,weights,k=0.03):
        self.weights=weights
        self.k=k
        self.zero()
        
    def zero(self):
        self.loss_sum=0.
        self.loss_count=0.
        self.lossf=0.
        
    def calc(self,y_pred,y_true,prefix=""):
        window=(y_true>=0).to(torch.float)
        loss = (F.binary_cross_entropy_with_logits(y_pred,y_true,reduction='none')*window*self.weights.expand_as(y_true)).mean()/(window.mean()+1e-5)
        self.lossf=self.lossf*(1-self.k)+loss*self.k
        self.loss_sum=self.loss_sum+loss*window.sum()
        self.loss_count=self.loss_count+window.sum()
        return({prefix+'mloss':self.lossf})    
        
    def calc_sums(self,prefix=""):
        return({prefix+'mloss_tot':self.loss_sum/self.loss_count})    



In [9]:
#features=(features-features.mean())/features.std()

In [10]:
class ResModel(nn.Module):
    def __init__(self,in_size):
        super(ResModel, self).__init__()
        self.dont_do_grad=[]
        self.conv2d1=torch.nn.Conv2d(1, 64, (9,in_size), padding=(4,0))
        self.bn0=torch.nn.BatchNorm1d(64)
        self.relu0=torch.nn.ReLU()
        self.conv1d1=torch.nn.Conv1d(64, 64, 7, padding=3)
        self.bn1=torch.nn.BatchNorm1d(64)
        self.relu1=torch.nn.ReLU()
        self.conv1d2=torch.nn.Conv1d(128, 64, 5, padding=2)
        self.bn2=torch.nn.BatchNorm1d(64)
        self.relu2=torch.nn.ReLU()
        self.conv1d3=torch.nn.Conv1d(192, 6, 3, padding=1)
        
        
    def forward(self, x):
        x = self.conv2d1(x.unsqueeze(1)).squeeze(-1)
        x = self.bn0(x)
        x0 = self.relu0(x)
        x = self.conv1d1(x0)
        x = self.bn1(x)
        x1 = self.relu1(x)
        x = torch.cat([x0,x1],1)
        x = self.conv1d2(x)
        x = self.bn2(x)
        x2 = self.relu2(x)
        x = torch.cat([x0,x1,x2],1)
        out = self.conv1d3(x).transpose(-1,-2)
        return out 
                    
    def no_grad(self):
        for param in self.parameters():
            param.requires_grad=False

    def do_grad(self):
        for n,p in self.named_parameters():
            p.requires_grad=  not any(nd in n for nd in self.dont_do_grad)


In [11]:
class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()
        self.dont_do_grad=[]
        self.conv2d1=torch.nn.Conv2d(1, 128, (5,2208), padding=(2,0))
        self.relu0=torch.nn.ReLU()
        self.conv1d1=torch.nn.Conv1d(128, 6, 1)

        
        
    def forward(self, x):
        x = self.conv2d1(x.unsqueeze(1)).squeeze(-1)
        x = self.relu0(x)
        out = self.conv1d1(x).transpose(-1,-2)
        return out 
                    
    def no_grad(self):
        for param in self.parameters():
            param.requires_grad=False

    def do_grad(self):
        for n,p in self.named_parameters():
            p.requires_grad=  not any(nd in n for nd in self.dont_do_grad)


In [12]:
class ResModelPool(nn.Module):
    def __init__(self,in_size):
        super(ResModelPool, self).__init__()
        self.dont_do_grad=[]
        self.conv2d1=torch.nn.Conv2d(1, 64, (9,in_size),stride=(1,in_size), padding=(4,0))
        self.bn0=torch.nn.BatchNorm1d(64)
#        self.relu0=torch.nn.ReLU()
        self.conv1d1=torch.nn.Conv1d(64, 64, 7, padding=3)
        self.bn1=torch.nn.BatchNorm1d(64)
        self.relu1=torch.nn.ReLU()
        self.conv1d2=torch.nn.Conv1d(128, 64, 5, padding=2)
        self.bn2=torch.nn.BatchNorm1d(64)
        self.relu2=torch.nn.ReLU()
        self.conv1d3=torch.nn.Conv1d(192, 6, 3, padding=1)
        
        
    def forward(self, x):
        x=x.unsqueeze(1)
        x = self.conv2d1(x)
        x=F.max_pool2d(x,kernel_size=(1,x.shape[-1])).squeeze(-1)        
        x0 = self.bn0(x)
#        x0 = self.relu0(x)
        x = self.conv1d1(x0)
        x = self.bn1(x)
        x1 = self.relu1(x)
        x = torch.cat([x0,x1],1)
        x = self.conv1d2(x)
        x = self.bn2(x)
        x2 = self.relu2(x)
        x = torch.cat([x0,x1,x2],1)
        out = self.conv1d3(x).transpose(-1,-2)
        return out 
                    
    def no_grad(self):
        for param in self.parameters():
            param.requires_grad=False

    def do_grad(self):
        for n,p in self.named_parameters():
            p.requires_grad=  not any(nd in n for nd in self.dont_do_grad)


In [14]:
multi=3

model_names=['se_resnext101_32x4d',
             'se_resnext101_32x4d',
             'Densenet161_3',
             'Densenet169_3',
             'se_resnet101',
             'se_resnet101',
             'se_resnet101']
types_train=['features_train_tta',
             'features_train_tta',
             'features_train_tta2',
             'features_train_tta2',
             'features_train_tta',
             'features_train_tta',
             'features_train_tta']
types_test=['features_test_tta',
            'features_test',
            'features_test2',
            'features_test2',
            'features_test',
            'features_test',
            'features_test']
versions=['classifier_splits',
          'new_splits',
          'classifier_splits',
          'classifier_splits',
          'classifier_splits',
          'new_splits',
          'new_splits_focal']
num_splits=[3,5,3,3,3,5,5]
seeds=[8153,432,8153,8153,8153,432,432]
for model_name,type_tr,type_ts,version_,n_splits,SEED in zip(model_names,types_train,types_test,versions,num_splits,seeds):
    pickle_file=open(outputs_dir+'PID_splits_{}_stage_2.pkl'.format(n_splits),'rb')
    split_sid,splits=pickle.load(pickle_file)
    pickle_file.close()
    for num_split in tqdm_notebook(range(n_splits)):
        weights = torch.tensor([1.,1.,1.,1.,1.,2.],dtype=torch.float)
        pred_list=[]
        print(model_name,version_,type_tr,type_ts,num_split) 
        pickle_file=open(outputs_dir+outputs_format.format(model_name,version_,type_tr,num_split),'rb')
        features=pickle.load(pickle_file)
        pickle_file.close()
        features.shape
        features=features.reshape(features.shape[0]//4,4,-1)
        features.shape
        pickle_file=open(outputs_dir+outputs_format.format(model_name,version_,type_ts,num_split),'rb')
        featurest=pickle.load(pickle_file)
        pickle_file.close()
        featurest.shape
        featurest=featurest.reshape(featurest.shape[0]//8,8,-1)
        featurest.shape
        features=torch.cat([features,featurest[:,:4]])
        features.shape
        dfm=pd.concat([old_train_df,old_test_df],sort=False,axis=0).reset_index(drop=True)[['PatientID']]
        atrain_df=dfm.merge(train_df,on='PatientID')
        split_train = atrain_df[atrain_df.PID.isin(set(split_sid[splits[num_split][0]]))].SeriesI.unique()
        split_validate =  atrain_df[atrain_df.PID.isin(set(split_sid[splits[num_split][1]]))].SeriesI.unique()
        print (split_train.shape,split_validate.shape)
        print (train_df[train_df.SeriesI.isin(set(split_train))].shape[0],
              train_df[train_df.SeriesI.isin(set(split_validate))].shape[0],)

        model=ResModelPool(features.shape[-1])
        version=version_+'_fullhead_resmodel_pool2_stage2_{}'.format(multi)

        model.load_state_dict(torch.load(models_dir+models_format.format(model_name,version,num_split),map_location=torch.device(device)))

        valid_dataset=FullHeadDataset(atrain_df,
                                      split_validate,
                                      features,
                                      'SeriesI',
                                      'ImagePositionZ',
                                      multi =3)

        win_dataset=FullHeadDataset(atrain_df,
                                      split_validate,
                                      features,
                                      'SeriesI',
                                      'ImagePositionZ',
                                       target_columns=hemorrhage_types)
        win_list=[]
        dl = D.DataLoader(win_dataset,batch_size=128,num_workers=16)
        for _,win in tqdm_notebook(dl):
            win_list.append(win.reshape(win.shape[0]*win.shape[1],-1))    
        wins = torch.cat(win_list,0).sum(1)>=0
        y_true = torch.cat(win_list,0)[wins].to(torch.float)
        print(train_df[train_df.PID.isin(set(split_sid[splits[num_split][1]]))].shape[0],wins.sum())
        for i in tqdm_notebook(range(32),leave=False):
            pr = model_run(model,valid_dataset,do_apex=False,batch_size=128)
            pred_list.append(pr.reshape(pr.shape[0]*pr.shape[1],-1)[wins])
        print(pred_list[0].shape)
        pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'OOF_pred',num_split),'wb')
        pickle.dump(pred_list,pickle_file,protocol=4)
        pickle_file.close()
        y_pred=torch.cat([p[None] for p in pred_list],0).mean(0)
        my_loss(y_pred,y_true,weights)


/home/reina/anaconda3/envs/RSNA/lib/python3.6/site-packages/ipykernel_launcher.py:34: ResourceWarning: unclosed file <_io.BufferedReader name='/media/hd/notebooks/data/RSNA/outputs/model_se_resnext101_32x4d_version_classifier_splits_type_features_train_tta_split_1.pkl'>


se_resnext101_32x4d classifier_splits features_train_tta features_test_tta 0


torch.Size([2697008, 256])

torch.Size([674252, 4, 256])

torch.Size([628360, 256])

...

(17376,) (4368,)
601783 151014


<All keys matched successfully>

151014 tensor(151014)


torch.Size([151014, 6])


tensor(0.0702)

In [40]:
multi=3

model_names=['se_resnext101_32x4d',
             'Densenet161_3',
             'Densenet169_3',
             'se_resnet101']
types_train=['features_train_tta',
             'features_train_tta2',
             'features_train_tta2',
             'features_train_tta']
types_test=['features_test',
            'features_test2',
            'features_test2',
            'features_test']
versions=['classifier_splits',
          'classifier_splits',
          'classifier_splits',
          'classifier_splits']
num_splits=[3,3,3,3]
seeds=[8153,8153,8153,8153]
# model_names=['se_resnext101_32x4d',
#              'se_resnet101',
#              'se_resnet101']
# types_train=['features_train_tta',
#              'features_train_tta',
#              'features_train_tta']
# types_test=['features_test',
#             'features_test',
#             'features_test']
# versions=['new_splits',
#           'new_splits',
#           'new_splits_focal']
# num_splits=[5,5,5]
# seeds=[432,432,432]
for model_name,type_tr,type_ts,version_,n_splits,SEED in zip(model_names,types_train,types_test,versions,num_splits,seeds):
    old_split_sid = old_train_df.PID.unique()
    old_split_sid.shape[0]
    old_splits=list(KFold(n_splits=n_splits,shuffle=True, random_state=SEED).split(old_split_sid))
    new_split_sid=np.array(list(set(train_df.PID.unique()).difference(old_train_df.PID.unique())))
    new_split_sid.shape[0]
    new_splits=list(KFold(n_splits=n_splits,shuffle=True, random_state=SEED).split(new_split_sid))
    split_sid=np.concatenate([old_split_sid,new_split_sid])
    print(split_sid.shape)
    splits=[]
    for i in range(len(old_splits)):
        splits.append((np.concatenate([old_splits[i][0],new_splits[i][0]+old_split_sid.shape[0]]),
                      np.concatenate([old_splits[i][1],new_splits[i][1]+old_split_sid.shape[0]])))
        print(splits[-1][0].shape,np.unique(splits[-1][0]).shape,splits[-1][1].shape,np.unique(splits[-1][1]).shape)

    for num_split in tqdm_notebook(range(n_splits)):
        

        print(train_df[train_df.PID.isin(set(split_sid[splits[num_split][1]]))].shape[0])
        version=version_+'_fullhead_resmodel_pool2_stage2_{}'.format(multi)
        pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'OOF_pred',num_split),'rb')
        tmp=pickle.load(pickle_file)
        pickle_file.close()
        print(tmp[0].shape)


17079

1859

(18938,)
(12625,) (12625,) (6313,) (6313,)
(12625,) (12625,) (6313,) (6313,)
(12626,) (12626,) (6312,) (6312,)


250629
torch.Size([250187, 6])
251768
torch.Size([251760, 6])
250400
torch.Size([250850, 6])


17079

...

250629
torch.Size([250170, 6])
251768
torch.Size([251852, 6])
250400
torch.Size([250775, 6])


17079

1859

(18938,)
(12625,) (12625,) (6313,) (6313,)
(12625,) (12625,) (6313,) (6313,)
(12626,) (12626,) (6312,) (6312,)


250629
torch.Size([250170, 6])
251768
torch.Size([251852, 6])
250400
torch.Size([250775, 6])


In [26]:
OOF_ids={}
SEED=8153
n_splits=3
pickle_file=open(outputs_dir+'PID_splits_{}_stage_2.pkl'.format(n_splits),'rb')
split_sid,splits=pickle.load(pickle_file)
pickle_file.close()
dfm=pd.concat([old_train_df,old_test_df],sort=False,axis=0).reset_index(drop=True)[['PatientID']]
atrain_df=dfm.merge(train_df,on='PatientID')

for i in range( n_splits):
    images_id_list=[]
    split_validate =  atrain_df[atrain_df.PID.isin(set(split_sid[splits[i][1]]))].SeriesI.unique()
    image_arr=atrain_df.PatientID.values
    ref_arr=atrain_df.SeriesI.values
    order_arr=atrain_df.ImagePositionZ.values
    for s in tqdm_notebook(split_validate):
        head_idx = np.where(ref_arr==s)[0]
        sorted_head_idx=head_idx[np.argsort(order_arr[head_idx])]
        images_id_list.append(image_arr[sorted_head_idx])
    image_ids=np.concatenate(images_id_list)
    print(image_ids.shape,atrain_df[atrain_df.PID.isin(set(split_sid[splits[i][1]]))].shape[0])
    OOF_ids[i]=image_ids


pickle_file=open(outputs_dir+'OOF_validation_image_ids_{}_stage2.pkl'.format(n_splits),'wb')
pickle.dump(OOF_ids,pickle_file,protocol=4)
pickle_file.close()


(250228,) 250228


(251792,) 251792


(250777,) 250777


In [27]:
n_splits=3
pickle_file=open('/media/hd/notebooks/data/RSNA/outputs/OOF_validation_image_ids_3_stage2.pkl','rb')
OOF_ids=pickle.load(pickle_file)
pickle_file.close()
pickle_file=open(outputs_dir+'PID_splits_{}_stage_2.pkl'.format(n_splits),'rb')
split_sid,splits=pickle.load(pickle_file)
pickle_file.close()


[OOF_ids[i].shape[0] for i in range(3)]
[train_df[train_df.PID.isin(set(split_sid[splits[i][1]]))].shape[0] for i in range(3)]
[len(set(train_df[train_df.PID.isin(set(split_sid[splits[i][1]]))].PatientID.values).difference(set(OOF_ids[i]))) for i in range(3)]

In [29]:
OOF_ids={}
SEED=432
n_splits=5
pickle_file=open(outputs_dir+'PID_splits_{}_stage_2.pkl'.format(n_splits),'rb')
split_sid,splits=pickle.load(pickle_file)
pickle_file.close()
dfm=pd.concat([old_train_df,old_test_df],sort=False,axis=0).reset_index(drop=True)[['PatientID']]
atrain_df=dfm.merge(train_df,on='PatientID')

for i in range( n_splits):
    images_id_list=[]
    split_validate =  atrain_df[atrain_df.PID.isin(set(split_sid[splits[i][1]]))].SeriesI.unique()
    image_arr=atrain_df.PatientID.values
    ref_arr=atrain_df.SeriesI.values
    order_arr=atrain_df.ImagePositionZ.values
    for s in tqdm_notebook(split_validate):
        head_idx = np.where(ref_arr==s)[0]
        sorted_head_idx=head_idx[np.argsort(order_arr[head_idx])]
        images_id_list.append(image_arr[sorted_head_idx])
    image_ids=np.concatenate(images_id_list)
    print(image_ids.shape,atrain_df[atrain_df.PID.isin(set(split_sid[splits[i][1]]))].shape[0])
    OOF_ids[i]=image_ids


pickle_file=open(outputs_dir+'OOF_validation_image_ids_{}_stage2.pkl'.format(n_splits),'wb')
pickle.dump(OOF_ids,pickle_file,protocol=4)
pickle_file.close()


(151238,) 151238


(147313,) 147313


(152135,) 152135


(151097,) 151097


(151014,) 151014


In [30]:
n_splits=5
pickle_file=open(outputs_dir+'OOF_validation_image_ids_{}_stage2.pkl'.format(n_splits),'rb')
OOF_ids=pickle.load(pickle_file)
pickle_file.close()
pickle_file=open(outputs_dir+'PID_splits_{}_stage_2.pkl'.format(n_splits),'rb')
split_sid,splits=pickle.load(pickle_file)
pickle_file.close()


[OOF_ids[i].shape[0] for i in range(n_splits)]
[train_df[train_df.PID.isin(set(split_sid[splits[i][1]]))].shape[0] for i in range(n_splits)]
[len(set(train_df[train_df.PID.isin(set(split_sid[splits[i][1]]))].PatientID.values).difference(set(OOF_ids[i]))) for i in range(n_splits)]

[151238, 147313, 152135, 151097, 151014]

[151238, 147313, 152135, 151097, 151014]

[0, 0, 0, 0, 0]

In [21]:
images_id_list=[]
split_all =  train_df.SeriesI.unique()
image_arr=train_df.PatientID.values
ref_arr=train_df.SeriesI.values
order_arr=train_df.ImagePositionZ.values
for s in tqdm_notebook(split_all):
    head_idx = np.where(ref_arr==s)[0]
    sorted_head_idx=head_idx[np.argsort(order_arr[head_idx])]
    images_id_list.append(image_arr[sorted_head_idx])
image_ids=np.concatenate(images_id_list)
image_ids.shape



(674252,)

In [15]:
pickle_file=open(outputs_dir+'ensemble_test_image_ids_stage2.pkl','wb')
pickle.dump(image_ids,pickle_file,protocol=4)
pickle_file.close()


In [23]:
pred_list=[]
for num_split in range(5):
    model_name,version =  'se_resnet101' , 'new_splits_fullhead_resmodel_pool2_stage2_3'
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'test_pred_ensemble',num_split),'rb')
    pred_list.extend(pickle.load(pickle_file))   
    pickle_file.close()
for num_split in range(5):
    model_name,version =  'se_resnext101_32x4d' , 'new_splits_fullhead_resmodel_pool2_stage2_3'
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'test_pred_ensemble',num_split),'rb')
    pred_list.extend(pickle.load(pickle_file))
    pickle_file.close()

In [24]:
len(pred_list)

320

In [26]:
len(pred_list)
pred=torch.sigmoid(torch.cat([p[None] for p in pred_list],0)).mean(0)
pred.shape

320

torch.Size([121232, 6])

In [30]:
pickle_file=open(outputs_dir+'ensemble_test_image_ids_stage2.pkl','rb')
image_ids = pickle.load(pickle_file)   
pickle_file.close()


In [31]:
submission_df=get_submission_ids(image_ids,pred,do_sigmoid=False)
submission_df.head(12)
submission_df.shape
sub_num='_stage2_3'
submission_df.to_csv('/media/hd/notebooks/data/RSNA/submissions/submission{}.csv'.format(sub_num),
                                                                  index=False, columns=['ID','Label'])


,ID,Label
0,ID_000000e27_any,0.086962
1,ID_000000e27_epidural,0.000254
2,ID_000000e27_intraparenchymal,0.005881
3,ID_000000e27_intraventricular,0.000138
4,ID_000000e27_subarachnoid,0.074892
5,ID_000000e27_subdural,0.013256
6,ID_000009146_any,0.000054
7,ID_000009146_epidural,0.000004
8,ID_000009146_intraparenchymal,0.000029
9,ID_000009146_intraventricular,0.000031


(727392, 2)

In [33]:
sub = pd.read_csv('/media/hd/notebooks/data/RSNA/submissions/submission_stage2_2.csv')

In [34]:
sub.Label=(sub.Label>0.5).astype(np.float)

In [35]:
df=sub.merge(submission_df,on='ID')
df.head()

,ID,Label_x,Label_y
0,ID_000000e27_any,0.0,0.086962
1,ID_000000e27_epidural,0.0,0.000254
2,ID_000000e27_intraparenchymal,0.0,0.005881
3,ID_000000e27_intraventricular,0.0,0.000138
4,ID_000000e27_subarachnoid,0.0,0.074892


In [36]:
F.binary_cross_entropy(torch.tensor(df.Label_y.values,dtype=torch.float),
                       torch.tensor(df.Label_x.values,dtype=torch.float))

tensor(0.0234)